# Домашнее задание №4

## Подготовительный этап

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import requests
import io
import re
from IPython.display import display
from tqdm import tqdm

## Задание №1

На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad средствами библиотеки **Pandas** сформируйте поле выводов по каждому анализирумомому параметру. Например, по показателю *pH* получен результат *8.4 единицы pH* при нормативе от *6 до 9 единиц pH*. Т.о. по данному показателю результат анализа в норме. Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, **6 >= pH >= 9**. В итоговом DataFrame столбец "Показатель" сделайте индексным.

In [2]:
# загрузим датафрейм с данными
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
data=pd.read_csv(io.StringIO(s.decode('UTF8')))

In [3]:
display(
    data
)

,<h1>Forbidden</h1><pre>Request ID: 2022-09-01-17-47-43-39A93363230E50AA</pre>


In [4]:
# загрузка не удалась, грузим из локального файла
data = pd.read_csv('data-20160608T1215-structure-20160608T1215.csv')

In [5]:
# посмотрим на данные
display(
    data
)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


В наших данных есть нюанс, который мешает их обработке автоматическим способом - в столбце **Результат анализа** наряду с числовыми значениями исследуемых показателей содержится текстовое значение `б/цвета`, которое нам нужно правильно интерпретировать, чтобы в дальнейшем обработать данные в автоматическом режиме. Для этого обратимся к ГОСТ 31868-2012 "Вода. Методы определения цветности". Из данного документа следует, что определение цветности происходит либо путём сравнения отобранной пробы с эталонной хромо-кобальтовой (или платиново-кобальтовой) шкалой или фотометрическим способом и даёт результат в так называемых "градусах цветности". Любое значение данного показателя, отличное от 0 означает присутствие некоторой цветовой компоненты в отобранной пробе. Т.к. в нашем случае имеется чёткое указание, что проба воды не имеет цвета, можно заменить данное значение на 0

In [6]:
# заменим индекс в таблице
data.set_index('Показатель', inplace=True)

In [7]:
# заменим значение
data.at['Цветность', 'Результат анализа'] = 0

In [8]:
display(data)

,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,в пределах 6-9
Запах,баллы,1,не более 2-3
Цветность,градусы,0,не более 30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24,не более 45
Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Хлориды (Cl),мг/дм3,200,не более 350


In [9]:
# приведём данные в столбце *Результат анализа* к типу float
data['Результат анализа'] = pd.to_numeric(data['Результат анализа'], downcast='float')

In [10]:
display(data)

,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.400,в пределах 6-9
Запах,баллы,1.000,не более 2-3
Цветность,градусы,0.000,не более 30
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24.000,не более 45
Фосфаты (P),мг/дм3,0.360,"не более 3,5"
Хлориды (Cl),мг/дм3,200.000,не более 350


In [11]:
# опишем функцию для сравнения с нормативом
def check_norm(row):
    # создаём словарь результатов
    results = {
        -1: 'Ниже нормы',
        0: 'В норме',
        1: 'Выше нормы'
    }

    # если задан диапазон допустимых значений
    if row['Норматив'].startswith('в пределах'):
        # отбираем границы диапазона
        n_min, n_max = [int(x) for x in re.findall(r'\d+', row['Норматив'])]
        # определяем, где лежит результат анализа - в диапазоне, меньше минимума или больше максимума
        check = -1 if row['Результат анализа'] < n_min else (0 if n_min <= row['Результат анализа'] <= n_max else 1)
    # если задано только максимальное значение
    else:
        # если максимальное значение записано в виде диапазона, берём его верхнюю границу
        if re.match(r'\D+\d+-\d+', row['Норматив']):
            n_max = int(re.findall(r'\d+', row['Норматив'])[-1])
        # если максимальное значение имеет мантиссу
        elif re.match(r'\D+\d+,\d+', row['Норматив']):
            n_max = float(re.search(r'\d+,\d+', row['Норматив']).group().replace(',', '.'))
        else:
            n_max = int(re.search(r'\d+', row['Норматив']).group())
        # определяем результат
        check = 0 if row['Результат анализа'] <= n_max else 1

    # возвращаем текстовое представление результата
    return results[check]

In [12]:
# сделаем вывод в автоматическом режиме
data['Вывод о показателе'] = data.apply(check_norm, axis=1)

In [13]:
# выведем на экран
display(data)

,Единица измерений,Результат анализа,Норматив,Вывод о показателе
Показатель,,,,
pH,единицы pH,8.400,в пределах 6-9,В норме
Запах,баллы,1.000,не более 2-3,В норме
Цветность,градусы,0.000,не более 30,В норме
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,В норме
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",В норме
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",В норме
Нитраты (по NO3),мг/дм3,24.000,не более 45,В норме
Фосфаты (P),мг/дм3,0.360,"не более 3,5",В норме
Хлориды (Cl),мг/дм3,200.000,не более 350,В норме


Как видно из полученных данных, по всем показателям, кроме двух (`Нефть` и `Окисляемость перманганатная`) вода в роднике соответствует нормативу

## Задание №2

В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?

Пусть апельсины будут нулями, а яблоки - единицами. Сформируем массив, иллюстрирующий наш набор фруктов:

In [14]:
fruits = np.concatenate((np.zeros(5), np.ones(4)), axis=0)

In [15]:
display(
    fruits
)

array([0., 0., 0., 0., 0., 1., 1., 1., 1.])

5 апельсинов и 4 яблока, всё по условию. Теперь проведём некоторое число экспериментов, где будем случайно выбирать 3 фрукта из набора, если они все три - апельсины (т.е. сумма элементов выборки равно `0`) - будем увеличивать счётчик значений. После окончания экспериментов разделим значение счётчика на число экспериментов и таким образом получим нашу экспериментальную вероятность.

In [16]:
# задаём число экспериментов и обнуляем счётчик
n_exp = 100000
count = 0

In [17]:
%%time
# проводим эксперименты
for i in tqdm(range(n_exp)):
    if np.random.choice(fruits, 3, replace=False).sum() == 0:
        count += 1

100%|██████████| 100000/100000 [00:04<00:00, 20317.91it/s]

CPU times: total: 5.02 s
Wall time: 4.94 s


In [18]:
# посчитаем нашу экспериментальную вероятность
print(f'Экспериментальная вероятность равна {count / n_exp:.3f}')

Экспериментальная вероятность равна 0.119


Уже при 100000 экспериментов удалось получить экспериментальную вероятность, совпадающую с аналитической

## Задание №3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?

В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

В данном задании нам важен не только набор результирующей выборки, но и порядок элементов в ней. Для простоты также обозначим стандартные детали нулями, а нестандарные - единицами. Далее будем перемешивать наш исходный набор в случайном порядке, и проверять, идут ли первые два элемента в ней в следующем порядке: 1 - 0. Если да - будем добавлять к счётчику единицу. Для получения экспериментальной вероятности разделим значение счётчика на число экспериментов

In [19]:
details = np.concatenate((np.zeros(7), np.ones(3)), axis=0)

In [20]:
display(details)

array([0., 0., 0., 0., 0., 0., 0., 1., 1., 1.])

7 стандартных, 3 нестандартных - всё как по заданию

In [21]:
# задаём число экспериментов
n_exp = 100000
count = 0

In [22]:
%%time
# проводим эксперименты
for i in tqdm(range(n_exp)):
    np.random.shuffle(details)
    if all(
        [
            details[0] == 1,
            details[1] == 0
        ]
    ):
        count += 1

100%|██████████| 100000/100000 [00:00<00:00, 142857.43it/s]

CPU times: total: 719 ms
Wall time: 703 ms


In [23]:
# посчитаем нашу экспериментальную вероятность
print(f'Экспериментальная вероятность равна {count / n_exp:.5f}')

Экспериментальная вероятность равна 0.23041


При 100000 экспериментов экспериментальная вероятность немного ниже, чем аналитическая. Попробуем увеличить число экспериментов в 100 раз

In [24]:
# увеличиваем число экспериментов
n_exp = 100 * n_exp
count = 0

In [25]:
%%time
# проводим эксперименты
for i in tqdm(range(n_exp)):
    np.random.shuffle(details)
    if all(
            [
                details[0] == 1,
                details[1] == 0
            ]
    ):
        count += 1

100%|██████████| 10000000/10000000 [01:22<00:00, 121781.82it/s]

CPU times: total: 1min 15s
Wall time: 1min 22s


In [26]:
# посчитаем нашу экспериментальную вероятность
print(f'Экспериментальная вероятность равна {count / n_exp:.5f}')

Экспериментальная вероятность равна 0.23336


In [27]:
# рассчитаем относительную погрешность
print(f'Относительная погрешность равна {(count / n_exp - 7 / 30) * 30 / 7:.2%}')

Относительная погрешность равна 0.01%


С такой относительной погрешностью пожно считать полученный результат приемлемым